In [15]:
from datascience import *
from prob140 import *     
import numpy as np
from scipy import special
import pickle

In [45]:
path_string_array = list('math')
path_string_array

['m', 'a', 't', 'h']

In [48]:
def new_transition_function(i, j):
    if i == 'm' and j == 'm':
        return 0.25
    elif i == 'm' and j == 'a':
        return 0.75
    elif i == 'm' and j == 't':
        return 0
    elif i == 'm' and j == 'h':
        return 0
    elif i == 'a' and j == 'a':
        return 0.25
    elif i == 'a' and j == 'm':
        return 0.25
    elif i == 'a' and j == 't':
        return 0.5
    elif i == 'a' and j == 'h':
        return 0
    elif i == 't' and j == 't':
        return 0.25
    elif i == 't' and j == 'a':
        return 0.5
    elif i == 't' and j == 'h':
        return 0.25
    elif i == 't' and j == 'm':
        return 0
    elif i == 'h' and j == 'h':
        return 0.25
    elif i == 'h' and j == 't':
        return 0.75
    elif i == 'h' and j == 'm':
        return 0
    elif i == 'h' and j == 'a':
        return 0 

In [49]:
math_walk = MarkovChain.from_transition_function(path_string_array, new_transition_function)
math_walk

,m,a,t,h
m,0.25,0.75,0.00,0.00
a,0.25,0.25,0.50,0.00
t,0.00,0.50,0.25,0.25
h,0.00,0.00,0.75,0.25


In [71]:
math_walk.prob_of_path('m', list('at'))

0.375

In [72]:
math_walk.transition_matrix(2)

,m,a,t,h
m,0.250,0.375,0.375,0.000
a,0.125,0.500,0.250,0.125
t,0.125,0.250,0.500,0.125
h,0.000,0.375,0.375,0.250


In [56]:
math_walk_p = math_walk.get_transition_matrix(2)
math_walk_p

array([[0.25, 0.75, 0.  , 0.  ],
       [0.25, 0.25, 0.5 , 0.  ],
       [0.  , 0.5 , 0.25, 0.25],
       [0.  , 0.  , 0.75, 0.25]])

In [73]:
np.linalg.matrix_power(math_walk_p, 2)

array([[0.25 , 0.375, 0.375, 0.   ],
       [0.125, 0.5  , 0.25 , 0.125],
       [0.125, 0.25 , 0.5  , 0.125],
       [0.   , 0.375, 0.375, 0.25 ]])

In [64]:
math_walk.transition_matrix(5)

,m,a,t,h
m,0.132812,0.398438,0.351562,0.117188
a,0.132812,0.367188,0.382812,0.117188
t,0.117188,0.382812,0.367188,0.132812
h,0.117188,0.351562,0.398438,0.132812


In [65]:
math_walk.transition_matrix(10)

,m,a,t,h
m,0.125488,0.375000,0.375000,0.124512
a,0.125000,0.375488,0.374512,0.125000
t,0.125000,0.374512,0.375488,0.125000
h,0.124512,0.375000,0.375000,0.125488


In [66]:
math_walk.transition_matrix(20)

,m,a,t,h
m,0.125,0.375,0.375,0.125
a,0.125,0.375,0.375,0.125
t,0.125,0.375,0.375,0.125
h,0.125,0.375,0.375,0.125


In [74]:
import itertools as it
from collections import Counter

def clean_string(string):
    """
    Cleans an input string by replacing all newlines with spaces, and then
    removing all letters not in *allowable_letters*
    """
    string = string.replace("\n"," ")
    return "".join([i for i in string.lower().strip() if i in allowable_letters])

def load_bigrams(text):
    """
    Takes a string which has already been cleaned, and returns a dictionary
    of conditional bigram probabilities
    
    cond_bigram[(a,b)] = P(X_{n+1} = b | X_{n} = a)
    
    Uses Laplace smoothing with size $1$, to remove zero transition probabilities
    """
    bigram_counter = Counter(list(it.product(allowable_letters, repeat=2)))
    gram_counter = Counter(allowable_letters*len(allowable_letters))
    for l1, l2 in zip(text,text[1:]):
        bigram_counter[(l1, l2)] += 1
        gram_counter[l1] += 1
    cond_bigram = {k:v/gram_counter[k[0]] for k,v in bigram_counter.items()}
    return cond_bigram
    
def bigram_from_file(filename):
    """
    Given a filename, this reads it, cleans it, and returns the conditional bigram
    """
    file_text = open(filename, encoding="utf8").read()
    file_text = clean_string(file_text)
    return load_bigrams(file_text)

def reverse_cipher(cipher):
    return {v:k for k, v in cipher.items()}

def print_differences(cipher1, cipher2):
    for k in cipher1:
        if cipher1[k] != cipher2[k]:
            print("%s: %s  %s"%(k,cipher1[k], cipher2[k]))
            
def num_errors(cipher, encoded_text, original_text):
    decoded = np.array(list(decode_text(encoded_text, cipher)))
    original = np.array(list(original_text))
    num_errors = np.count_nonzero(decoded != original)
    return num_errors

def get_secret_text(student_id):
    with open('Lab05_data/secret_strings.p', 'rb') as f:
        texts = pickle.load(f)
    return texts[student_id % len(texts)]

In [145]:
simple_alphabet = list('ehnt')

In [146]:
jane_bigrams = bigram_from_file("1342-0.txt")

In [147]:
def jane_transition(a, b):
    if (a, b) in jane_bigrams:
        return jane_bigrams[(a, b)]
    return 0

jane_bigram_mc = MarkovChain.from_transition_function(simple_alphabet, jane_transition)
jane_bigram_mc

c:\users\seany\appdata\local\programs\python\python36\lib\site-packages\prob140\markov_chains.py:21: UserWarning: Transition probabilities don't sum to 1.
  warnings.warn('Transition probabilities don\'t sum to 1.')


,e,h,n,t
e,0.027507,0.002174,0.089070,0.036820
h,0.433861,0.000144,0.000202,0.027338
n,0.104807,0.001006,0.020306,0.090977
t,0.097812,0.292171,0.001596,0.025012


In [135]:
jane_bigram_mc.prob_of_path('t', ['h', 'e', 'n'])

0.011290692800076626

In [136]:
jane_bigram_mc.prob_of_path('n', ['e', 'h', 't'])

6.229418162272677e-06

In [87]:
allowable_letters = list("abcdefghijklmnopqrstuvwxyz ")


In [88]:
jane_bigrams = bigram_from_file("1342-0.txt")

In [156]:
def jane_transition(a, b):
    if (a, b) in jane_bigrams:
        return jane_bigrams[(a, b)]
    return 0

jane_bigram_mc = MarkovChain.from_transition_function(allowable_letters, jane_transition)
jane_bigram_mc

,a,b,c,d,e,f,g,h,i,j,...,r,s,t,u,v,w,x,y,z,
a,0.000187,0.036531,0.027135,0.057215,0.000210,0.011078,0.021035,0.000164,0.037653,0.000023,...,0.099822,0.114617,0.136002,0.012107,0.032067,0.008157,0.000280,0.028748,0.000561,0.053405
b,0.031200,0.000852,0.000106,0.000213,0.456927,0.000106,0.000106,0.000852,0.046427,0.019806,...,0.034714,0.016718,0.011926,0.121925,0.000213,0.000106,0.000106,0.073794,0.000106,0.001597
c,0.101879,0.000141,0.024375,0.000283,0.181998,0.000071,0.000071,0.167585,0.039000,0.000071,...,0.023456,0.000565,0.101809,0.026565,0.000071,0.000071,0.000071,0.036032,0.000071,0.007418
d,0.048730,0.000393,0.000087,0.009440,0.114680,0.001136,0.005463,0.000568,0.088283,0.000087,...,0.010708,0.018793,0.000961,0.010533,0.004414,0.001005,0.000044,0.017831,0.000044,0.610900
e,0.044409,0.001108,0.025290,0.077779,0.027507,0.009959,0.004685,0.002174,0.015261,0.000715,...,0.163455,0.052474,0.036820,0.000337,0.020072,0.005470,0.010464,0.016439,0.000014,0.315295
f,0.063648,0.000161,0.000564,0.000564,0.092330,0.047454,0.000242,0.000725,0.068321,0.000081,...,0.069771,0.000645,0.034322,0.030696,0.000081,0.000242,0.000081,0.003706,0.000081,0.386320
g,0.070283,0.008690,0.000191,0.000573,0.126337,0.000286,0.003438,0.145817,0.052617,0.000095,...,0.067609,0.021963,0.009549,0.023109,0.000095,0.000191,0.000095,0.002387,0.000095,0.351891
h,0.184964,0.001557,0.000029,0.000288,0.433861,0.000346,0.000029,0.000144,0.131874,0.000029,...,0.004816,0.002192,0.027338,0.007959,0.000029,0.000144,0.000029,0.002278,0.000029,0.121262
i,0.018879,0.008696,0.040067,0.035116,0.038938,0.019418,0.022983,0.000154,0.000077,0.000026,...,0.038015,0.127947,0.127972,0.000590,0.023804,0.000051,0.001462,0.000026,0.019777,0.053174
j,0.305221,0.001004,0.001004,0.001004,0.324297,0.001004,0.001004,0.001004,0.002008,0.001004,...,0.001004,0.001004,0.001004,0.192771,0.001004,0.001004,0.001004,0.001004,0.001004,0.001004


In [149]:
simple_decoder_1 = {'e':'n', 'h':'h', 'n':'t', 't':'e'}
proposed_original_text_1 = decode_text('nhte', simple_decoder_1)
proposed_original_text_1

'then'

In [150]:
simple_decoder_2 = {'e':'t', 'h':'e', 'n':'n', 't':'h'}
proposed_original_text_2 = decode_text('nhte', simple_decoder_2)
proposed_original_text_2

'neht'

In [151]:
simple_decoder_3 = {'e':'h', 'h':'n', 'n':'e', 't':'t'}
proposed_original_text_3 = decode_text('nhte', simple_decoder_3)
proposed_original_text_3

'enth'

In [152]:
score_1 = log_prob_path(proposed_original_text_1, jane_bigram_mc)
score_1

-4.483776538657503

In [153]:
score_2 = log_prob_path(proposed_original_text_2, jane_bigram_mc)
score_2

-11.986227622419932

In [154]:
score_3 = log_prob_path(proposed_original_text_3, jane_bigram_mc)
score_3

-6.0458929064298665

In [157]:
decoder_letters = np.array(list("abcdefghijklmnopqrstuvwxyz")) 
# We don't operate on spaces

identity_decoder = \
    {letter:letter for letter in decoder_letters}

# Random starting decoder
def random_decoder():
    """ Random decoder """
    new_letters = decoder_letters.copy()
    np.random.shuffle(new_letters)
    return {orig:new for orig ,new in zip(decoder_letters, new_letters)}

In [158]:
def decode_text(string, decoder):
    new_string = ""
    
    for char in string:
        if char in decoder:
            new_letter = decoder.get(char)
        else:
            new_letter = char
        
        # Now we append the letter to the back of the new string
        new_string = new_string + new_letter
        
    return new_string

In [159]:
np.math.factorial(26)**2

162644002617632464507038883409628607021056000000000000

In [160]:
def generate_proposed_decoder(decoder):
    new_decoder = decoder.copy()
    
    letters = np.random.choice(
        decoder_letters, 2, replace=False)
    
    letter1 = letters[0]
    letter2 = letters[1]
    
    new_value_of_letter1 = new_decoder.get(letter2)
    new_value_of_letter2 = new_decoder.get(letter1)

    # This code replaces the value of letter1 and letter2 
    # with new_value_of_letter1 and new_value_of_letter2
    new_decoder[letter1] = new_value_of_letter1
    new_decoder[letter2] = new_value_of_letter2
    return new_decoder

In [161]:
def log_prob_path(string, bigram_mc):
    start = string[0]
    
    path = list(string[1:])
    
    return bigram_mc.log_prob_of_path(start, path)

In [162]:
def log_score(string, decoder, bigrams):
    return log_prob_path(decode_text(string,decoder), bigrams)

In [28]:
secret_text = open('message_secret.txt').read()
secret_text

'lv xae jivr alwxigt iz xae qigjk ivjt u zeq reveguxlivw ause heev rguvxek xae gije iz kezevklvr zgeekid lv lxw aiyg iz duoldyd kuvreg l ki vix waglvn zgid xalw gewmivwlhljlxtl qejcide lx l ki vix hejlese xaux uvt iz yw qiyjk eocauvre mjucew qlxa uvt ixaeg meimje ig uvt ixaeg reveguxliv xae evegrt xae zulxa xae kesixliv qalca qe hglvr xi xalw evkeusig qljj jlrax iyg ciyvxgt uvk ujj qai wegse lxuvk xae rjiq zgid xaux zlge cuv xgyjt jlrax xae qigjk uvk wi dt zejjiq udeglcuvw uwn vix qaux tiyg ciyvxgt cuv ki zig tiyuwn qaux tiy cuv ki zig tiyg ciyvxgt dt zejjiq clxlpevw iz xae qigjk uwn vix qaux udeglcu qljj ki zig tiy hyx qaux xirexaeg qe cuv ki zig xae zgeekid iz duv      wiygce fiav z nevvektw lvuyryguj wmeeca'

In [163]:
import time

def p_coin(p):
    """
    Flips a coin that comes up heads with probability p
    and returns 1 if Heads, 0 if Tails
    """
    return np.random.random() < p


def metropolis(string_to_decode, bigrams, reps):
    decoder = random_decoder() # Starting decoder 
    best_decoder = decoder
    best_score = log_score(
        string_to_decode,
        best_decoder,
        bigrams
    )
    last_score = best_score
    for rep in np.arange(reps):
        # This will print out our progress
        if rep*10%reps == 0: # Repeat every 10%
            time.sleep(.01)
            decoded_text = decode_text(
                string_to_decode,
                best_decoder
            )[:40]
            print('Score: %.00f \t Guess: %s'%(best_score, decoded_text))  
        proposed_decoder = generate_proposed_decoder(decoder)
        log_s_orig = log_score(string_to_decode, decoder, bigrams)
        log_s_new = log_score(string_to_decode, proposed_decoder, bigrams)
        # If better than before or p-coin flip works
        if log_s_new > log_s_orig or p_coin(np.e**(log_s_new-log_s_orig)): 
            decoder = proposed_decoder
            
            if log_s_new > best_score:
                best_decoder = decoder
                best_score = log_s_new
    return best_decoder

In [32]:
ogtext = """
I have, myself, full confidence that if all do their duty, if nothing is neglected, 
and if the best arrangements are made, as they are being made, we shall prove ourselves 
once again able to defend our Island home, to ride out the storm of war, and to outlive 
the menace of tyranny, if necessary for years, if necessary alone. At any rate, that is 
what we are going to try to do. That is the resolve of His Majesty’s Government-every man 
of them. That is the will of Parliament and the nation. 
"""
original_string = clean_string(ogtext)
encoding_cipher = random_decoder()
string_to_decode = decode_text(original_string,encoding_cipher)
string_to_decode

'q ztwn lsinab bdaa cerbqhnrcn yzty qb taa he yznqf hdys qb reyzqrx qi rnxancynh  trh qb yzn mniy tfftrxnlnryi tfn lthn ti yzns tfn mnqrx lthn jn iztaa vfewn edfinawni  ercn txtqr tman ye hnbnrh edf qiatrh zeln ye fqhn edy yzn iyefl eb jtf trh ye edyaqwn  yzn lnrtcn eb ysftrrs qb rncniitfs bef sntfi qb rncniitfs taern ty trs ftyn yzty qi  jzty jn tfn xeqrx ye yfs ye he yzty qi yzn fnieawn eb zqi ltgniysi xewnfrlnrynwnfs ltr  eb yznl yzty qi yzn jqaa eb vtfaqtlnry trh yzn rtyqer'

In [164]:
ogtext = """
I have a dream that one day this nation will rise up and live out the true meaning of its creed: 'We hold these 
truths to be self-evident; that all men are created equal."I have a dream that one day on the red hills of Georgia 
the sons of former slaves and the sons of former slave owners will be able to sit together at the table of brotherhood.
I have a dream that one day even the state of Mississippi, a state sweltering with the heat of injustice, sweltering 
with the heat of oppression, will be transformed into an oasis of freedom and justice.
"""
original_string = clean_string(ogtext)
encoding_cipher = random_decoder()
string_to_decode = decode_text(original_string,encoding_cipher)
string_to_decode

'u nhfm h rlmhp qnhq gem rhv qnuy ehquge zubb luym ws her bufm gwq qnm qlwm pmheuec ga uqy ilmmr zm ngbr qnmym  qlwqny qg jm ymbamfurmeq qnhq hbb pme hlm ilmhqmr mtwhbu nhfm h rlmhp qnhq gem rhv ge qnm lmr nubby ga cmglcuh  qnm ygey ga aglpml ybhfmy her qnm ygey ga aglpml ybhfm gzemly zubb jm hjbm qg yuq qgcmqnml hq qnm qhjbm ga jlgqnmlnggr u nhfm h rlmhp qnhq gem rhv mfme qnm yqhqm ga puyyuyyussu h yqhqm yzmbqmluec zuqn qnm nmhq ga uekwyquim yzmbqmluec  zuqn qnm nmhq ga gsslmyyuge zubb jm qlheyaglpmr ueqg he ghyuy ga almmrgp her kwyquim'

In [168]:
new_decoder = metropolis(string_to_decode, wp_bigram_mc, 10000)

Score: -3046 	 Guess: d mgjv g kwvgn cmgc slv kgt cmdi lgcdsl 
Score: -1352 	 Guess: i have a greap that one gay this nation 
Score: -1288 	 Guess: i have a dream that one day this nation 
Score: -1284 	 Guess: i have a dream that one day this nation 
Score: -1283 	 Guess: i have a dream that one day this nation 
Score: -1283 	 Guess: i have a dream that one day this nation 
Score: -1283 	 Guess: i have a dream that one day this nation 
Score: -1283 	 Guess: i have a dream that one day this nation 
Score: -1283 	 Guess: i have a dream that one day this nation 
Score: -1283 	 Guess: i have a dream that one day this nation 


In [169]:
decode_text(string_to_decode, new_decoder)

'i have a dream that one day this nation will rise up and live out the true meaning of its creed we hold these  truths to be selfevident that all men are created equali have a dream that one day on the red hills of georgia  the sons of former slaves and the sons of former slave owners will be able to sit together at the table of brotherhood i have a dream that one day even the state of mississippi a state sweltering with the heat of injustice sweltering  with the heat of oppression will be transformed into an oasis of freedom and justice'

In [35]:
secret_decoder = metropolis(secret_text, wp_bigram_mc, 10000)
decode_text(secret_text, secret_decoder)

Score: -3639 	 Guess: kh jex olhu ekdjltn lr jex mltof lhon i 
Score: -1917 	 Guess: in dle mong litdory of dle worms onmy a 
Score: -1757 	 Guess: in the mong history of the wormd onmy a 
Score: -1700 	 Guess: in the long history of the world only a 
Score: -1700 	 Guess: in the long history of the world only a 
Score: -1700 	 Guess: in the long history of the world only a 
Score: -1700 	 Guess: in the long history of the world only a 
Score: -1700 	 Guess: in the long history of the world only a 
Score: -1700 	 Guess: in the long history of the world only a 
Score: -1700 	 Guess: in the long history of the world only a 


'in the long history of the world only a few generations have been granted the role of defending freedom in its hour of maximum danger i do not shrink from this responsibilityi welcome it i do not believe that any of us would exchange places with any other people or any other generation the energy the faith the devotion which we bring to this endeavor will light our country and all who serve itand the glow from that fire can truly light the world and so my fellow americans ask not what your country can do for youask what you can do for your country my fellow citizens of the world ask not what america will do for you but what together we can do for the freedom of man      source john f kennedys inaugural speech'